In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
import os
import pickle
import openpyxl
import dill as pickle

In [2]:
TOP = os.getcwd().replace('notebooks', '')
raw_dir = TOP + 'data/raw/'
interim_dir = TOP + 'data/interim/'
external_dir = TOP + 'data/external/'
processed_dir = TOP + 'data/processed/'
figures_dir = TOP + 'reports/figures/'

In [3]:
toxval = pd.read_excel(processed_dir+'CCD-Batch-Search_2024-09-03_10_31_46_toxvalids.xlsx', sheet_name = 'Main Data')

In [4]:
smi_dict = (toxval
 .assign(smiles = lambda x: x['SMILES'].str.replace('  ', '', regex = True).str.replace(r'^$', '', regex=True).replace(' ', np.nan))
 .dropna(subset = ['smiles'])
 .set_index('DTXSID')['smiles'].to_dict()
)


In [5]:
toxval =  (toxval
 .assign(smiles = lambda x: x['SMILES'].str.replace('  ', '', regex = True).str.replace(r'^$', '', regex=True).replace(' ', np.nan))
 .dropna(subset = ['smiles'])
 )

In [6]:
smi_dict2 = {k:Chem.MolFromSmiles(v) for k,v in zip(toxval['DTXSID'], toxval['smiles'])}

[10:28:53] WARNING: not removing hydrogen atom without neighbors
[10:28:53] WARNING: not removing hydrogen atom without neighbors
[10:28:53] Explicit valence for atom # 2 Cl, 2, is greater than permitted
[10:28:53] WARNING: not removing hydrogen atom without neighbors
[10:28:53] WARNING: not removing hydrogen atom without neighbors
[10:28:53] WARNING: not removing hydrogen atom without neighbors
[10:28:53] WARNING: not removing hydrogen atom without neighbors
[10:28:53] Explicit valence for atom # 2 Cl, 2, is greater than permitted
[10:28:53] WARNING: not removing hydrogen atom without neighbors
[10:28:53] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5


In [7]:
s = list(smi_dict2.keys())

In [8]:
toxval2 = (toxval
.query('DTXSID in @s')
)

In [9]:
df = pd.read_excel(interim_dir+'tsca_categorisation_290624_wmappingdict.xlsx', index_col = [0])

In [10]:
toxval2 = (toxval2
 .merge(df[['dtxsid',  'NCC',  'group']], left_on = 'DTXSID', right_on = 'dtxsid', how = 'left')
 
)

In [11]:
tsca_df = df[['dtxsid',  'NCC', 'ClassyFire', 'group']]

In [12]:
import pickle
filename = 'finalized_rf_model_mgrn.sav'
rf_model = pickle.load(open(external_dir+filename, 'rb'))

In [13]:
my_dict = pickle.load(open(external_dir+'category_mapping_mgrn.pkl', 'rb'))

In [14]:
class MoleculeProcessor:
    def __init__(self, smiles, dtx):
        self.mol = Chem.MolFromSmiles(smiles)
        self.dtx = dtx

    def get_morgan_fingerprint(self, radius = 3, nBits=2048):
        mgrn_df = pd.DataFrame([np.array(AllChem.GetMorganFingerprintAsBitVect(self.mol,radius,nBits))] )
        mgrn_df.columns = ['mrgn_%d'%i for i in mgrn_df.columns]
        mgrn_df.index = [self.dtx]
        return mgrn_df
    

In [15]:
cm = MoleculeProcessor( 'CCCC',123)

In [16]:
test = cm.get_morgan_fingerprint()

In [17]:
X=[]
for i, row in toxval2.iterrows():
    try:
        a = MoleculeProcessor(row['smiles'], row['DTXSID']).get_morgan_fingerprint()
    except Exception as error:
        print(error)
    else:
        X.append(a)

[10:29:39] WARNING: not removing hydrogen atom without neighbors
[10:29:39] WARNING: not removing hydrogen atom without neighbors
[10:29:39] Explicit valence for atom # 2 Cl, 2, is greater than permitted
[10:29:39] WARNING: not removing hydrogen atom without neighbors
[10:29:39] WARNING: not removing hydrogen atom without neighbors


Python argument types in
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(NoneType, int, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)
Python argument types in
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(NoneType, int, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)


[10:29:50] WARNING: not removing hydrogen atom without neighbors
[10:29:50] WARNING: not removing hydrogen atom without neighbors
[10:29:57] Explicit valence for atom # 2 Cl, 2, is greater than permitted


Python argument types in
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(NoneType, int, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)
Python argument types in
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(NoneType, int, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)
Python argument types in
    rdkit.Chem.rdMolDescriptors

[10:30:24] WARNING: not removing hydrogen atom without neighbors


Python argument types in
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(NoneType, int, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)


[10:30:37] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5


Python argument types in
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(NoneType, int, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)


In [18]:
X_df = pd.concat(X)

In [19]:
class CategoryPrediction:
    def __init__(self, rev_dict, rf_model):

        self.rev_dict = rev_dict
        self.rf_model = rf_model

    def make_prediction(self, df):
        '''
        Make prediction using the RF model and convert back into the original terminal categories
        '''
        pred = self.rf_model.predict(df)[0]
        return self.rev_dict[pred]


In [20]:
rev_dict = {v:k for k, v in my_dict.items()}

In [21]:
catpred = CategoryPrediction(rev_dict, rf_model )

In [22]:
#catpred.make_prediction(test)

In [23]:
results = {}
for e in X_df.index.tolist():
     results[e] = catpred.make_prediction(X_df[X_df.index==e])

In [24]:
results_df = pd.DataFrame(list(results.items()), columns = ['dtxsid', 'category'])
results_df.head()

,dtxsid,category
0,DTXSID001004378,"('Carboxylic acids and derivatives', 2.0)"
1,DTXSID001005033,"('Carboxylic acids and derivatives', 5.0)"
2,DTXSID001009930,"('Piperidines', nan)"
3,DTXSID001014486,"('Homogeneous metal compounds', nan)"
4,DTXSID001016113,"('Indoles and derivatives', nan)"


In [25]:
s1 = X_df.index.tolist()

In [26]:
toxval3 = (toxval2
 .query('DTXSID in @s1')
 .merge(results_df, left_on = 'DTXSID', right_on = 'dtxsid')
.drop(['dtxsid_y'], axis = 1)
.assign(dtxsid = lambda x: x['dtxsid_x'])
           .drop(['dtxsid','dtxsid_x', 'INPUT', 'FOUND_BY'], axis = 1)
)

In [27]:
toxval3 = (toxval3
 .assign(dtxsid = lambda x: x['DTXSID'])
 .drop(['DTXSID'], axis = 1)
 .merge(pd.read_csv(processed_dir+'derived_oral_noncancer_PODs_Aurisano_approach_030924.csv')[['dtxsid', 'pPOD']], on = 'dtxsid', how = 'left')
.pipe(lambda df: df[[df.columns[-2]] + df.columns[:-2].tolist() + [df.columns[-1]]])  # Reorder penultimate column
)

In [28]:
toxval3.to_csv(processed_dir+'toxval_tsca_cats_pred.csv')

In [29]:
import pymongo
from pymongo import MongoClient

In [30]:
client = MongoClient('mongodb://removed:27017/')

In [31]:
DB = client.genra_dev_v5

In [32]:
smi_dict = {k:v for k,v in zip(toxval3['dtxsid'], toxval3['SMILES'])}

In [33]:
#smi_dict
len(smi_dict)

4159

In [34]:
loael_logp = [e for e in DB.physprop.find({'dsstox_sid': {'$in': list(smi_dict.keys())}}, {"dsstox_sid":1, "predicted_props.OPERA_LogP" : 1, "_id":0})]

In [35]:
loael_logp[101]

{'predicted_props': {}, 'dsstox_sid': 'DTXSID0020234'}

In [36]:
logp_dict = {record['dsstox_sid']: record.get('predicted_props',{}).get('OPERA_LogP', [None])[0] for record in loael_logp}

In [37]:
#logp_dict

In [38]:
loael_ws = [e for e in DB.physprop.find({'dsstox_sid': {'$in': list(smi_dict.keys())}}, {"dsstox_sid":1, "predicted_props.OPERA_WS" : 1, "_id":0})]

In [39]:
ws_dict = {record['dsstox_sid']: record.get('predicted_props',{}).get('OPERA_WS', [None])[0] for record in loael_ws}

In [40]:
loael_weight = [e for e in DB.compounds.find({'dsstox_sid': {'$in': list(smi_dict.keys())}}, {"dsstox_sid":1, "mol_weight" : 1, "_id":0})]

In [41]:
weight_dict={record['dsstox_sid']:record['mol_weight'] for record in loael_weight}

In [42]:
sids=set(logp_dict.keys())&set(ws_dict.keys())&set(weight_dict.keys())&set(smi_dict.keys())

In [43]:
records=[]
for sid in sids:
    records.append({'dsstox_sid':sid,'smiles':smi_dict[sid],'logp':logp_dict[sid],'ws':ws_dict[sid],'mol_weight':weight_dict[sid],'mol':Chem.MolFromSmiles(smi_dict[sid])})
records=[record for record in records if record['mol']]

[10:34:24] WARNING: not removing hydrogen atom without neighbors
[10:34:24] WARNING: not removing hydrogen atom without neighbors
[10:34:24] WARNING: not removing hydrogen atom without neighbors
[10:34:24] WARNING: not removing hydrogen atom without neighbors


In [44]:
import dill as pickle
with open(external_dir+"epa_cats.pkl", "rb") as output_file:
     new_tests = pickle.load(output_file)

In [45]:
def convert_ppb(x): #OPERA results stored as mol/L
    ws=x['ws']
    mol_weight=x['mol_weight']
    return ws*mol_weight*10**6

In [46]:
import operator as op
op_dict={
    'GreaterThan': op.gt,
    'GreaterThanOrEqualTo': op.ge,
    'LessThan': op.lt,
    'LessThanOrEqualTo': op.le
}
prop_dict={
    'log Kow':'logp',
    'Molecular Weight':'mol_weight',
    'Molecular weight':'mol_weight',
    'Water Solubility': 'ws'
}

In [47]:
records[6]

{'dsstox_sid': 'DTXSID0022016',
 'smiles': 'C1CN2CCN1CC2',
 'logp': -0.14817,
 'ws': 5.54046,
 'mol_weight': 112.176,
 'mol': <rdkit.Chem.rdchem.Mol at 0x791faee63370>}

In [48]:
filtered_records = [record for record in records if all(value is not None for value in record.values())]

In [66]:
#len(filtered_records)

In [49]:
#filtered_records[6]

In [50]:
import math, re
for record in filtered_records:
    #print(record)
    epa_categories=sorted([category for category,test in new_tests.items() if test(record)])
   
    if 'Neutral Organics' in epa_categories and len(epa_categories)>1:
        epa_categories.remove('Neutral Organics')
    record['categories']=tuple(epa_categories)

In [51]:
toxval3.columns

Index(['dtxsid', 'PREFERRED_NAME', 'CASRN', 'SMILES', 'smiles', 'NCC', 'group',
       'category', 'pPOD'],
      dtype='object')

In [52]:
(toxval3
 .merge(pd.DataFrame(filtered_records)[['dsstox_sid', 'categories']], left_on = 'dtxsid', right_on = 'dsstox_sid', how = 'left')
)

,dtxsid,PREFERRED_NAME,CASRN,SMILES,smiles,NCC,group,category,pPOD,dsstox_sid,categories
0,DTXSID001004378,4-tert-Butylcyclohexyl prop-2-enoate,84100-23-2,CC(C)(C)C1CCC(CC1)OC(=O)C=C,CC(C)(C)C1CCC(CC1)OC(=O)C=C,"('Acrylates/Methacrylates (Acute toxicity)', '...","('Carboxylic acids and derivatives', 2.0)","('Carboxylic acids and derivatives', 2.0)",1.279506,NaN,NaN
1,DTXSID001005033,"Bis(2-ethylhexyl) cyclohexane-1,4-dicarboxylate",84731-70-4,CCCCC(CC)COC(=O)C1CCC(CC1)C(=O)OCC(CC)CCCC,CCCCC(CC)COC(=O)C1CCC(CC1)C(=O)OCC(CC)CCCC,NaN,NaN,"('Carboxylic acids and derivatives', 5.0)",2.180879,NaN,NaN
2,DTXSID001009930,Thialdine,638-17-5,C[C@H]1N[C@@H](C)S[C@@H](C)S1,C[C@H]1N[C@@H](C)S[C@@H](C)S1,NaN,NaN,"('Piperidines', nan)",0.348332,NaN,NaN
3,DTXSID001014486,Calcium molybdate(VI),7789-82-4,[O--].[O--].[O--].[O--].[Ca++].[Mo+6],[O--].[O--].[O--].[O--].[Ca++].[Mo+6],NaN,NaN,"('Homogeneous metal compounds', nan)",1.039907,NaN,NaN
4,DTXSID001016113,Eletriptan hydrobromide,177834-92-3,Br.CN1CCC[C@@H]1CC1=CNC2=CC=C(CCS(=O)(=O)C3=CC...,Br.CN1CCC[C@@H]1CC1=CNC2=CC=C(CCS(=O)(=O)C3=CC...,NaN,NaN,"('Indoles and derivatives', nan)",0.667420,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4154,DTXSID90968887,Methyl 3-sulfanylbutanoate,54051-19-3,COC(=O)CC(C)S,COC(=O)CC(C)S,NaN,NaN,"('Carboxylic acids and derivatives', 1.0)",-1.841999,NaN,NaN
4155,DTXSID90976780,3-[(2-Methylfuran-3-yl)sulfanyl]butan-2-one,61295-44-1,CC(SC1=C(C)OC=C1)C(C)=O,CC(SC1=C(C)OC=C1)C(C)=O,NaN,NaN,"('Heteroaromatic compounds', nan)",-0.044963,NaN,NaN
4156,DTXSID90978461,Heptane-2-thiol,628-00-2,CCCCCC(C)S,CCCCCC(C)S,NaN,NaN,"('Organosulfur compounds', 4.0)",-1.841999,NaN,NaN
4157,DTXSID90990466,"3-Amino-N-(8-hydroxy-4,6-disulfonaphthalen-1-y...",70239-77-9,NC1=CC=CC(=C1)C(O)=NC1=C2C(O)=CC(=CC2=C(C=C1)S...,NC1=CC=CC(=C1)C(O)=NC1=C2C(O)=CC(=CC2=C(C=C1)S...,NaN,NaN,"('Naphthalenes', 3.0)",1.606971,NaN,NaN


In [53]:
toxval3 = (toxval3
 .merge(
pd.DataFrame(filtered_records), left_on = 'dtxsid', right_on = 'dsstox_sid', how = 'left')
 
)
        

In [57]:
toxval4 =(toxval3
 .drop(['smiles_y', 'dsstox_sid', 'logp', 'ws', 'mol_weight', 'mol', 'NCC'], axis = 1)
 .rename(columns = {'smiles_x': 'smiles', 'categories': 'NCC'})
)

In [59]:
toxval4.NCC.value_counts()

NCC
()                                                                                                                                                                1615
(Neutral Organics,)                                                                                                                                                432
(Esters (Acute toxicity),)                                                                                                                                         309
(Phenols (Acute toxicity),)                                                                                                                                        170
(Anilines (Acute toxicity),)                                                                                                                                        71
                                                                                                                                                                 

In [60]:
toxval4.to_csv(processed_dir+'toxval4_predcats.csv')

In [63]:
opera_df = pd.read_csv(external_dir+'toxval_31024-sdf_OPERA2.9Pred.csv')

In [80]:
opera_df = (opera_df
 .rename(columns = {'MoleculeID': 'dsstox_sid', 'LogP_pred': 'logp', 'MolWeight': 'mol_weight', 'LogWS_pred': 'ws'}
        )
 [['dsstox_sid', 'mol_weight', 'logp','ws']]
 .merge(pd.read_csv(external_dir+'toxval_31024_Summary_file.csv')[['Molecule name', 'Canonical_QSARr']], left_on = 'dsstox_sid', right_on = 'Molecule name', how = 'left')
 .rename(columns = {'Canonical_QSARr': 'smiles'})
 .drop(['Molecule name'], axis = 1)
)

In [82]:

opera_df['mol'] = [Chem.MolFromSmiles(e) for e in opera_df['smiles']]

[15:18:29] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 9 10 11 13 14 15


In [88]:
opera_df = opera_df.dropna(subset = ['logp'])

In [95]:
opera_df = opera_df.dropna()

In [96]:
test_dict = opera_df.to_dict('records')

In [97]:
opera_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3817 entries, 0 to 3818
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   dsstox_sid  3817 non-null   object 
 1   mol_weight  3817 non-null   float64
 2   logp        3817 non-null   float64
 3   ws          3817 non-null   float64
 4   smiles      3817 non-null   object 
 5   mol         3817 non-null   object 
dtypes: float64(3), object(3)
memory usage: 208.7+ KB


In [98]:
for record in test_dict:
    #print(record)
    epa_categories=sorted([category for category,test in new_tests.items() if test(record)])
    if 'Neutral Organics' in epa_categories and len(epa_categories)>1:
        epa_categories.remove('Neutral Organics')
    record['categories']=tuple(epa_categories)

In [103]:
toxval4 = (toxval4
 .merge(pd.DataFrame(test_dict)[['dsstox_sid', 'categories']], left_on = 'dtxsid', right_on = 'dsstox_sid', how = 'left')
 .drop(['dsstox_sid', 'NCC'], axis = 1)
 .rename(columns = {'categories': 'NCC'})
)

In [104]:
toxval4.to_csv(processed_dir+'toxval4_all_predcats.csv')

In [108]:
toxval4.NCC.value_counts(dropna = False)

NCC
()                                                                         1911
(Neutral Organics,)                                                         538
NaN                                                                         342
(Esters (Acute toxicity),)                                                  334
(Phenols (Acute toxicity),)                                                 198
                                                                           ... 
(Aldehydes (Chronic toxicity), Esters (Chronic toxicity))                     1
(Acrylates/Methacrylates (Chronic toxicity),)                                 1
(Alkoxysilanes, Substituted Triazines (Acute toxicity))                       1
(Anilines (Acute toxicity), Polynitroaromatics (Acute toxicity))              1
(Anilines (Acute toxicity), Cationic (quaternary ammonium) surfactants)       1
Name: count, Length: 95, dtype: int64

In [110]:
toxval4

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4159 entries, 0 to 4158
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   dtxsid          4159 non-null   object 
 1   PREFERRED_NAME  4159 non-null   object 
 2   CASRN           4159 non-null   object 
 3   SMILES          4159 non-null   object 
 4   smiles          4159 non-null   object 
 5   group           1981 non-null   object 
 6   category        4159 non-null   object 
 7   pPOD            4159 non-null   float64
 8   NCC             3817 non-null   object 
dtypes: float64(1), object(8)
memory usage: 292.6+ KB


In [111]:
X1=[]
for i, row in toxval4.iterrows():
    try:
        a = MoleculeProcessor(row['smiles'], row['dtxsid']).get_morgan_fingerprint()
    except Exception as error:
        print(error)
    else:
        X1.append(a)

[18:52:42] WARNING: not removing hydrogen atom without neighbors
[18:52:42] WARNING: not removing hydrogen atom without neighbors
[18:52:42] WARNING: not removing hydrogen atom without neighbors
[18:52:42] WARNING: not removing hydrogen atom without neighbors
[18:52:53] WARNING: not removing hydrogen atom without neighbors
[18:52:53] WARNING: not removing hydrogen atom without neighbors
[18:53:28] WARNING: not removing hydrogen atom without neighbors


In [113]:
X1 = pd.concat(X1)

In [149]:
test = MoleculeProcessor('CC1(C)COC(C)(OC1)C1C=CC=CC=1', 'test').get_morgan_fingerprint()

In [150]:
test

,mrgn_0,mrgn_1,mrgn_2,mrgn_3,mrgn_4,mrgn_5,mrgn_6,mrgn_7,mrgn_8,mrgn_9,...,mrgn_2038,mrgn_2039,mrgn_2040,mrgn_2041,mrgn_2042,mrgn_2043,mrgn_2044,mrgn_2045,mrgn_2046,mrgn_2047
test,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [119]:
toxval4.columns

Index(['dtxsid', 'PREFERRED_NAME', 'CASRN', 'SMILES', 'smiles', 'group',
       'category', 'pPOD', 'NCC'],
      dtype='object')

In [120]:
toxval4[['dtxsid', 'SMILES', 'category', 'pPOD', 'NCC']]

,dtxsid,SMILES,category,pPOD,NCC
0,DTXSID001004378,CC(C)(C)C1CCC(CC1)OC(=O)C=C,"('Carboxylic acids and derivatives', 2.0)",1.279506,"(Acrylates/Methacrylates (Chronic toxicity), E..."
1,DTXSID001005033,CCCCC(CC)COC(=O)C1CCC(CC1)C(=O)OCC(CC)CCCC,"('Carboxylic acids and derivatives', 5.0)",2.180879,"(Esters (Chronic toxicity),)"
2,DTXSID001009930,C[C@H]1N[C@@H](C)S[C@@H](C)S1,"('Piperidines', nan)",0.348332,()
3,DTXSID001014486,[O--].[O--].[O--].[O--].[Ca++].[Mo+6],"('Homogeneous metal compounds', nan)",1.039907,NaN
4,DTXSID001016113,Br.CN1CCC[C@@H]1CC1=CNC2=CC=C(CCS(=O)(=O)C3=CC...,"('Indoles and derivatives', nan)",0.667420,()
...,...,...,...,...,...
4154,DTXSID90968887,COC(=O)CC(C)S,"('Carboxylic acids and derivatives', 1.0)",-1.841999,"(Esters (Acute toxicity), Thiols (Acute toxici..."
4155,DTXSID90976780,CC(SC1=C(C)OC=C1)C(C)=O,"('Heteroaromatic compounds', nan)",-0.044963,()
4156,DTXSID90978461,CCCCCC(C)S,"('Organosulfur compounds', 4.0)",-1.841999,"(Thiols (Acute toxicity),)"
4157,DTXSID90990466,NC1=CC=CC(=C1)C(O)=NC1=C2C(O)=CC(=CC2=C(C=C1)S...,"('Naphthalenes', 3.0)",1.606971,"(Anilines (Acute toxicity),)"


In [133]:
(pd.concat([X1, toxval4[['dtxsid', 'SMILES', 'category', 'pPOD', 'NCC']].set_index('dtxsid')], axis =1)
 .to_csv(external_dir+'X1.csv')
)

In [137]:
X1 = pd.read_csv(external_dir+'X1.csv', index_col = [0])

In [147]:
X1.loc[:, :'mrgn_2047']

,mrgn_0,mrgn_1,mrgn_2,mrgn_3,mrgn_4,mrgn_5,mrgn_6,mrgn_7,mrgn_8,mrgn_9,...,mrgn_2038,mrgn_2039,mrgn_2040,mrgn_2041,mrgn_2042,mrgn_2043,mrgn_2044,mrgn_2045,mrgn_2046,mrgn_2047
DTXSID001004378,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID001005033,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID001009930,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID001014486,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID001016113,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DTXSID90968887,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID90976780,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID90978461,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID90990466,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [139]:
from sklearn.neighbors import NearestNeighbors

In [148]:
nbrs = NearestNeighbors(n_neighbors=10, metric='jaccard').fit(X1.loc[:, :'mrgn_2047'])

In [151]:
distances, indices = nbrs.kneighbors(test)

/home/grace/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [177]:
sims = 1-distances

In [153]:
indices

array([[1996,  875, 3031, 3395, 2178, 2992, 3394, 2595, 3179,  953]])

In [163]:
neighbor_indices_df = pd.DataFrame(indices)

# Map the positional indices to the actual row labels from X1
# 'X1.index' contains the actual row labels
neighbor_labels_df = neighbor_indices_df.applymap(lambda idx: X1.index[idx])


/tmp/ipykernel_304485/141390605.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  neighbor_labels_df = neighbor_indices_df.applymap(lambda idx: X1.index[idx])


In [192]:
(pd.concat([neighbor_labels_df.T, pd.DataFrame(sims).T], axis =1)
 .set_axis( ['analogue_id', 'Jaccard'], axis = 1)
)

,analogue_id,Jaccard
0,DTXSID4051401,0.347826
1,DTXSID201311373,0.276596
2,DTXSID7023510,0.254902
3,DTXSID8020591,0.250000
4,DTXSID5021122,0.240000
5,DTXSID7021360,0.235294
6,DTXSID8020541,0.234043
7,DTXSID6021828,0.230769
8,DTXSID7040360,0.225806
9,DTXSID2021446,0.222222


In [183]:
neighbor_labels_df.T

,0
0,DTXSID4051401
1,DTXSID201311373
2,DTXSID7023510
3,DTXSID8020591
4,DTXSID5021122
5,DTXSID7021360
6,DTXSID8020541
7,DTXSID6021828
8,DTXSID7040360
9,DTXSID2021446


In [165]:
FP2 = pd.concat([X1.loc[:, :'mrgn_2047'], test])

In [167]:
from scipy.spatial.distance import pdist, squareform

In [168]:
D_mgrn = pd.DataFrame(squareform(pdist(FP2, 'jaccard')), columns = FP2.index, index = FP2.index)

In [169]:
S_mgrn = 1-D_mgrn
S_mgrn.head()

,DTXSID001004378,DTXSID001005033,DTXSID001009930,DTXSID001014486,DTXSID001016113,DTXSID001019355,DTXSID001019793,DTXSID001019836,DTXSID001020128,DTXSID001020930,...,DTXSID90925652,DTXSID90961393,DTXSID90961439,DTXSID90968685,DTXSID90968887,DTXSID90976780,DTXSID90978461,DTXSID90990466,DTXSID90998205,test
DTXSID001004378,1.000000,0.126984,0.042553,0.0,0.059406,0.032787,0.059701,0.088889,0.049180,0.041096,...,0.050000,0.163934,0.054545,0.021277,0.104167,0.064516,0.019608,0.044444,0.022472,0.033333
DTXSID001005033,0.126984,1.000000,0.058824,0.0,0.056604,0.096774,0.151515,0.072917,0.150000,0.065789,...,0.062500,0.169231,0.050000,0.104167,0.137255,0.075758,0.187500,0.031250,0.021277,0.030769
DTXSID001009930,0.042553,0.058824,1.000000,0.0,0.034483,0.045455,0.080000,0.012500,0.021739,0.017241,...,0.022222,0.058824,0.051282,0.033333,0.028571,0.020833,0.060606,0.000000,0.027778,0.022727
DTXSID001014486,0.000000,0.000000,0.000000,1.0,0.000000,0.031250,0.000000,0.014925,0.000000,0.000000,...,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.015873,0.000000,0.000000
DTXSID001016113,0.059406,0.056604,0.034483,0.0,1.000000,0.029703,0.056604,0.069231,0.039604,0.083333,...,0.106383,0.066667,0.031250,0.034884,0.032967,0.059406,0.033333,0.106557,0.073171,0.095745


In [170]:
def nearest_neighbour(S, target, n = 10):
    t_m =S[target].values
    ids_chem = {i:x for i, x in enumerate(S.index)}
    temp_chem = np.argpartition(-t_m, n)
    chem_nn = temp_chem[:n]
    nn_chem = [ids_chem[x] for x in chem_nn]
    mydict = {k:v for k,v in list(zip(nn_chem,t_m[chem_nn]))}
    df = pd.DataFrame(mydict.items(), columns = ['dtxsid', 'Jaccard']).sort_values(by = 'Jaccard', ascending = False)
    return df

In [172]:
nearest_neighbour(S_mgrn, 'test', n= 10)

,dtxsid,Jaccard
1,test,1.000000
0,DTXSID4051401,0.347826
2,DTXSID201311373,0.276596
3,DTXSID7023510,0.254902
4,DTXSID8020591,0.250000
7,DTXSID5021122,0.240000
8,DTXSID7021360,0.235294
6,DTXSID8020541,0.234043
5,DTXSID6021828,0.230769
9,DTXSID7040360,0.225806


In [196]:
df[['dtxsid', 'group', 'screenable', 'final_picks']]

,dtxsid,group,screenable,final_picks
0,DTXSID4063036,"('Acetylides', nan)",1,NaN
1,DTXSID30870753,"('Acetylides', nan)",0,NaN
2,DTXSID7062374,"('Acetylides', nan)",0,NaN
3,DTXSID9061097,"('Acetylides', nan)",1,1.0
4,DTXSID1061233,"('Acetylides', nan)",0,NaN
...,...,...,...,...
14242,DTXSID7070391,"('Triphenyl compounds', nan)",1,NaN
14243,DTXSID70889405,"('Triphenyl compounds', nan)",1,1.0
14244,DTXSID90886794,"('Triphenyl compounds', nan)",1,NaN
14245,DTXSID3073052,"('Triphenyl compounds', nan)",1,NaN
